## Dash

**Reference:** https://realpython.com/python-dash/

**Dash** is an open source framework for building data visualization interfaces
1. **Flask** supplies the web server functionality.
2. **React.js** renders the user interface of the web page.
3. **Plotly.js** generates the charts used in your application.

_But you don’t have to worry about making all these technologies work together. Dash will do that for you._

**Popular projects using Dash**
- [A dashboard to analyze trading positions in real-time](https://dash.gallery/dash-web-trader/)
- [A visualization of millions of Uber rides](https://dash.gallery/dash-uber-rides-demo/)
- [An interactive financial report](https://dash.gallery/dash-financial-report/)

**Things to know before moving on**
1. Python graphing libraries
2. HTML
3. CSS

In [1]:
# install dash library
!pip install dash==1.13.3

**Process of building a Dash application**
1. Define the looks of your application using app's **layout**
2. Use **callbacks** to determine which parts of your app are interactive and what they react to

### Import libraries

In [6]:
import dash # initialize your application
import dash_core_components as dcc # allows you to create interactive components like graphs, dropdowns, or date ranges
import dash_html_components as html # access HTML tags
import pandas as pd
import numpy as np

from dash.dependencies import Output, Input

### Data

In [3]:
data = pd.read_csv("csvs\\avocado.csv") # read avocado csv file
data = data.query("type == 'conventional' and region == 'Albany'") # filter the query
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d") # convert the date into a date format
data.sort_values("Date", inplace=True) # sort the datetime in ascending order

# adding custom style
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
                "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
app.title = "Avocado Analytics: Understand Your Avocados!"

### App layout

In [4]:
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="🥑", className="header-emoji"),
                html.H1(
                    children="Avocado Analytics", className="header-title"
                ),
                html.P(
                    children="Analyze the behavior of avocado prices"
                    " and the number of avocados sold in the US"
                    " between 2015 and 2018",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Region", className="menu-title"),
                        dcc.Dropdown(
                            id="region-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in np.sort(data.region.unique())
                            ],
                            value="Albany",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Type", className="menu-title"),
                        dcc.Dropdown(
                            id="type-filter",
                            options=[
                                {"label": avocado_type, "value": avocado_type}
                                for avocado_type in data.type.unique()
                            ],
                            value="organic",
                            clearable=False,
                            searchable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range", className="menu-title"
                        ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=data.Date.min().date(),
                            max_date_allowed=data.Date.max().date(),
                            start_date=data.Date.min().date(),
                            end_date=data.Date.max().date(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="price-chart",
                        config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="volume-chart",
                        config={"displayModeBar": False},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    ]
)

**Two sets of components**
1. **Dash HTML Components** provides you with Python wrappers for HTML elements. For example, you could use this library to create elements such as paragraphs, headings, or lists.
2. **Dash Core Components** provides you with Python abstractions for creating interactive user interfaces. You can use it to create interactive elements such as graphs, sliders, or dropdowns.

### Run the application

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=False)

## Style your Dash application

> Dash provides you with a lot of flexibility to customize the look of your application

**Two ways to style your components**
1. Using the `style` argument of individual components
2. Providing an external CSS file

#### Why not the first option

Using the `style` argument to customize your dashboard is straightforward. \
But the downside of using the style argument is that it doesn’t scale well as your codebase grows. \
If your dashboard has multiple components that you want to look the same, then you’ll end up repeating a lot of your code. Instead, you can use a custom CSS file.

If you want to include your own local CSS or JavaScript files, then you need to create a folder called `assets/` in the root directory of your project and save the files you want to add there. By default, Dash automatically serves any file included in `assets/`.

## Add interactivity to your Dash application using callbacks

> You can link components with elements of your app that you want to update. If a user interacts with an input component like a dropdown or a range slider, then the output, such as a graph, will react automatically to the changes in the input.

In [7]:
@app.callback(
    [Output("price-chart", "figure"), Output("volume-chart", "figure")],
    [
        Input("region-filter", "value"),
        Input("type-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_charts(region, avocado_type, start_date, end_date):
    mask = (
        (data.region == region)
        & (data.type == avocado_type)
        & (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    filtered_data = data.loc[mask, :]
    price_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["AveragePrice"],
                "type": "lines",
                "hovertemplate": "$%{y:.2f}<extra></extra>",
            },
        ],
        "layout": {
            "title": {
                "text": "Average Price of Avocados",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#17B897"],
        },
    }

    volume_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Total Volume"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Avocados Sold", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#E12D39"],
        },
    }
    return price_chart_figure, volume_chart_figure

### Run

In [8]:
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2021 17:26:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 17:26:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 17:26:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 17:26:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 17:26:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 17:27:04] "POST /_dash-update-component HTTP/1.1" 200 -
